In [1]:
import pandas as pd
import pandahouse
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import swifter
from scipy import stats
%matplotlib inline

In [2]:
connection = {'host': 'https://clickhouse.lab.karpov.courses',
'database':'simulator_20241020',
'user':'student',
'password':'dpo_python_2020'
}

In [6]:
q = """
SELECT user_id, exp_group, SUM(action = 'like') AS likes, SUM(action = 'view') AS views
FROM {db}.feed_actions
WHERE toDate(time) >= '2024-09-20' AND toDate(time) <= '2024-09-26' AND exp_group IN (0,1)
GROUP BY user_id, exp_group
"""

In [8]:
df_0_1 = pandahouse.read_clickhouse(q, connection=connection)

In [10]:
df_0_1['ctr'] = df_0_1.likes / df_0_1.views

In [11]:
df_0_1['ctr'].mean()

0.21942631658330572

In [16]:
q1 = """
SELECT user_id, exp_group, SUM(action = 'like') AS likes, SUM(action = 'view') AS views
FROM {db}.feed_actions
WHERE toDate(time) >= '2024-09-27' AND toDate(time) <= '2024-10-03' AND exp_group IN (0,3)
GROUP BY user_id, exp_group
"""

In [17]:
df_0_3 = pandahouse.read_clickhouse(q1, connection=connection)

In [18]:
df_0_3['ctr'] = df_0_3.likes / df_0_3.views

In [19]:
stats.ttest_ind(df_0_3[df_0_3.exp_group == 0].ctr, df_0_3[df_0_3.exp_group == 3].ctr, equal_var=False)

Ttest_indResult(statistic=-13.935320516755848, pvalue=6.216047483055657e-44)

In [20]:
CTR_control = df_0_3[df_0_3.exp_group == 0].ctr.mean()

In [21]:
df_0_3['lin_likes'] = df_0_3.likes - CTR_control * df_0_3.views

In [23]:
stats.ttest_ind(df_0_3[df_0_3.exp_group == 0].lin_likes, df_0_3[df_0_3.exp_group == 3].lin_likes, equal_var=False)

Ttest_indResult(statistic=-16.137763029444965, pvalue=3.242948328128896e-58)

In [24]:
q2 = """
SELECT user_id, exp_group, SUM(action = 'like') AS likes, SUM(action = 'view') AS views
FROM {db}.feed_actions
WHERE toDate(time) >= '2024-09-27' AND toDate(time) <= '2024-10-03' AND exp_group IN (1,2)
GROUP BY user_id, exp_group
"""

In [25]:
df_1_2 = pandahouse.read_clickhouse(q2, connection=connection)

In [26]:
df_1_2['ctr'] = df_1_2.likes / df_1_2.views

In [27]:
stats.ttest_ind(df_1_2[df_1_2.exp_group == 1].ctr, df_1_2[df_1_2.exp_group == 2].ctr, equal_var=False)

Ttest_indResult(statistic=0.405149191311259, pvalue=0.6853733311407633)

In [28]:
df_1_2['lin_likes'] = df_1_2.likes - CTR_control * df_1_2.views

In [29]:
stats.ttest_ind(df_1_2[df_1_2.exp_group == 1].lin_likes, df_1_2[df_1_2.exp_group == 2].lin_likes, equal_var=False)

Ttest_indResult(statistic=5.877448545126082, pvalue=4.257657684372571e-09)